# Интро и полезная литература

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [1]:
cd ..

/Users/n.barsukov/WebstormProjects/ml-bug-detector


Заимпортируем всё, что нам пригодится в дальнейшем

In [2]:
from shared.helpers.word2VecPreprocessing import filter_tokens_fo_Word2Vec

Также внешние библиотеки подключим сразу

In [3]:
from os import walk
import pandas as pd
import json

from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
from tqdm.notebook import tqdm

# Готовим данные перед построением модели

In [4]:
# папка, в которой лежат нужные нам json
FOLDER_WITH_JSONS = 'data'

# как начинается имя json, в которых лежат токенизированные скрипты
TOKENIZED_JSON_START_WITH_PREFIX = 'tokenized-scripts'

Так как токенизация была разбита на микрозадачи (чтобы хватило мощностей компьютера), то и json у нас много теперь.

In [5]:
# filenames - Список всех файлов в интересуемой нами директории
fdirpath, dirnames, filenames = list(walk(FOLDER_WITH_JSONS))[0]

tokenized_scripts_jsons_paths = list(
    filter(
        lambda script_name: script_name.startswith(TOKENIZED_JSON_START_WITH_PREFIX),
        filenames
    )
)

Можно было бы пытаться объединить их все в одну, но опять же тут в зависимости от мощностей компьютера, может просто не хватить оперативки держать столько данных в памяти.

Благо word2Vec может учить постепенно: вначале на одних данных, потом "доучить" другим набором данных.

Поэтому вначале учим на первом наборе данных, а потом доучиваем оставшимися постепенно.

In [6]:
def transform_json_file_to_train_data(path_to_json_file):
    tokenized_scripts_dic = {}
    
    tokenizedScriptsJson = open(path_to_json_file)
    raw_tokenized_scripts_dictionary = json.load(tokenizedScriptsJson)

    for fileName in raw_tokenized_scripts_dictionary.keys():
        tokenized_scripts_dic[fileName] = filter_tokens_fo_Word2Vec(raw_tokenized_scripts_dictionary[fileName])
        
    return list(tokenized_scripts_dic.values())

In [7]:
first_train_json_path = f"{FOLDER_WITH_JSONS}/{tokenized_scripts_jsons_paths[0]}"

tokenized_scripts_list_of_lists = transform_json_file_to_train_data(first_train_json_path)

# Build Word2Vec model

Теперь бы нужно построить модель и настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [8]:
word2VecModel = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=tokenized_scripts_list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=5,
    
    # финальный размер словаря
    max_final_vocab=10000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 5, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

Смотрим, какие наиболее близкие вектора будут к функции с названием **setTimeout** (это встроенная функция в js, которая на вход принимает функцию первым аргументом, которую нужно исполнить через заданное кол-во времени в миллисекундах, которые будут переданы вторым аргументом).

Число напротив каждого слова - процент близости векторов.

In [9]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('prevOp', 0.9845616817474365),
 ('fireEvent', 0.9843124747276306),
 ('oldInnerHTML', 0.9841935634613037),
 ('saving', 0.9839743375778198),
 ('mouseTarget', 0.9833625555038452),
 ('onComplete', 0.9829936623573303),
 ('click', 0.9828275442123413),
 ('findElement', 0.9824248552322388),
 ("'close'", 0.9823454022407532),
 ('uniqueId', 0.9817233681678772),
 ('refreshClasses', 0.9814270734786987),
 ('togglePlaceholderText', 0.9813440442085266),
 ('closing', 0.9808827638626099),
 ('trigger', 0.9799845218658447),
 ('addListener', 0.9793043732643127),
 ('onSelectionChange', 0.9792879819869995),
 ('updateFull', 0.9789943099021912),
 ('showPrintMargin', 0.9785706996917725),
 ('attachEvents', 0.9777292609214783),
 ('disable', 0.9770629405975342)]

Видим, что модель на маленьком кусочке данных ($\frac{1}{43}$ от всех токенов) показывает крайне странные результаты. Да, в каждом слове можно "натянуть" смысл, но это все будет не то.

Посмотрим лучше, какие будут результаты у модели, когда она "доучиться" на остальных данных.

# Resume learning

API библиотеки gensim предоставил возможность продолжать обучение модели на новых данных

In [10]:
additional_data_jsons = tokenized_scripts_jsons_paths[1:]

for jsonPathIndex in tqdm(range(len(additional_data_jsons))):
    try:
        new_data = transform_json_file_to_train_data(f"{FOLDER_WITH_JSONS}/{additional_data_jsons[jsonPathIndex]}")
    except:
        print('Не получилось расспарсить json:', additional_data_jsons[jsonPathIndex])
        continue
    
    word2VecModel.build_vocab(new_data, update=True)
    word2VecModel.train(new_data, total_examples=word2VecModel.corpus_count, epochs=word2VecModel.epochs)

In [11]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('setInterval', 0.715908944606781),
 ('clearTimeout', 0.6965270042419434),
 ('clearInterval', 0.6727496385574341),
 ('detectingSelectAll', 0.6090414524078369),
 ('timer', 0.5855937004089355),
 ('2000', 0.5547860860824585),
 ('5000', 0.5504839420318604),
 ('timerId', 0.5426981449127197),
 ('scrollTo', 0.5400852560997009),
 ('timeout_id', 0.5396533012390137),
 ('requiredParams', 0.5387997627258301),
 ('delay', 0.5387283563613892),
 ('focus', 0.5380741357803345),
 ('fadeOut', 0.5342597961425781),
 ('300', 0.5268067717552185),
 ('getZForResolution', 0.523263692855835),
 ('_delay', 0.5221475958824158),
 ('timeout', 0.5221250057220459),
 ('interval', 0.5156717300415039),
 ('longPressTimer', 0.5145896077156067)]

Результаты стали гораздо более впечатляющими!

- clearTimeout - обрывает запланированный вызов функции
- timeout - через какое время исполнить функцию (один из агрументов всегда)

# Сохранение и загрузка модели

In [12]:
word2VecModel.save('word2Vec/word2VecModel')

In [13]:
from gensim.models.word2vec import Word2Vec
word2VecModel = Word2Vec.load('word2Vec/word2VecModel')

In [14]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('setInterval', 0.715908944606781),
 ('clearTimeout', 0.6965270042419434),
 ('clearInterval', 0.6727496385574341),
 ('detectingSelectAll', 0.6090414524078369),
 ('timer', 0.5855937004089355),
 ('2000', 0.5547860860824585),
 ('5000', 0.5504839420318604),
 ('timerId', 0.5426981449127197),
 ('scrollTo', 0.5400852560997009),
 ('timeout_id', 0.5396533012390137),
 ('requiredParams', 0.5387997627258301),
 ('delay', 0.5387283563613892),
 ('focus', 0.5380741357803345),
 ('fadeOut', 0.5342597961425781),
 ('300', 0.5268067717552185),
 ('getZForResolution', 0.523263692855835),
 ('_delay', 0.5221475958824158),
 ('timeout', 0.5221250057220459),
 ('interval', 0.5156717300415039),
 ('longPressTimer', 0.5145896077156067)]

In [15]:
print('В нашем словаре теперь', len(word2VecModel.wv.vocab), 'слов')

В нашем словаре теперь 128891 слов
